<a href="https://colab.research.google.com/github/allanpichardo/audio-generation-autoencoder/blob/main/Audio_Generation_with_Autoencoders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kapre==0.3.4

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 184kB 19.1MB/s 
     |████████████████████████████████| 61kB 10.4MB/s 
  Created wheel for kapre: filename=kapre-0.3.4-cp36-none-any.whl size=20608 sha256=4939da14af6b2bc58db37e88c378f87b1e9811da67089d747931fe04b27fd5c2
  Stored in directory: /root/.cache/pip/wheels/f8/9e/dc/cc6f4989c58eee2913abac4fab60723ff91aa2a06c6de4280f
Successfully built kapre
  Created wheel for librosa: filename=librosa-0.8.0-cp36-none-any.whl size=201375 sha256=12c39f74fa5d6228912471c63df802154ad63a81dfd2445b2edb500e70754ab8
  Stored in directory: /root/.cache/pip/wheels/ee/10/1e/382bb4369e189938d5c02e06d10c651817da8d485bfd1647c9
Successfully built librosa
  Found existing installation: librosa 0.6.3
    Uninstalling librosa-0.6.3:
      Successfully uninstalled librosa-0.6.3
  Found existing installation: kapre 0.1.3.1
    Uninstalling kapre-0.1.

In [2]:
import tensorflow as tf

In [3]:
MODELS_URL = "https://raw.githubusercontent.com/allanpichardo/audio-generation-autoencoder/main/models.py"
SAMPLES_DRIVE_ID = "1dHQqjYClND3fLHjM--cOWQh_ucS480QI"

In [4]:
tf.keras.utils.get_file(
    "/content/models.py",
    MODELS_URL
)

!gdown --id 1dHQqjYClND3fLHjM--cOWQh_ucS480QI
!mkdir /content/samples
!unzip /content/samples.zip -d /content

8192/7447 [=================================] - 0s 0us/step
Downloading...
From: https://drive.google.com/uc?id=1dHQqjYClND3fLHjM--cOWQh_ucS480QI
To: /content/samples.zip
1.39GB [00:09, 140MB/s]
Archive:  /content/samples.zip
   creating: /content/samples/monopoly/
  inflating: /content/samples/monopoly/DRUMS-CryingTom.wav  
  inflating: /content/samples/monopoly/SYNTH-BigLeadzzz.wav  
  inflating: /content/samples/monopoly/BASS-ResoThump.wav  
  inflating: /content/samples/monopoly/SYNTH-BeautySoftChordHiFiltered.wav  
  inflating: /content/samples/monopoly/FXCarpenterModulation.wav  
  inflating: /content/samples/monopoly/DRUMS-NoiseBrush.wav  
  inflating: /content/samples/monopoly/ARP-DreaminCentralPark.wav  
  inflating: /content/samples/monopoly/ARP-FluxioRhythm.wav  
  inflating: /content/samples/monopoly/ARP-008TeleFax.wav  
  inflating: /content/samples/monopoly/SYNTH-KoreaFiltered.wav  
  inflating: /content/samples/monopoly/CHORD-ResWitchBell.wav  
  inflating: /content/samp

In [5]:
from models import SoundSequence, get_models_cnn

In [6]:
encoder, decoder, autoencoder = get_models_cnn(latent_dim=8)
encoder.summary()
decoder.summary()

autoencoder.compile(optimizer=tf.keras.optimizers.Adam())

Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 1025, 2 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 256, 1025, 32 608         input_1[0][0]                    
__________________________________________________________________________________________________
re_lu (ReLU)                    (None, 256, 1025, 32 0           conv2d[0][0]                     
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 256, 1025, 32 128         re_lu[0][0]                      
____________________________________________________________________________________________

In [7]:
path = '/content/samples'
sr = 22050
duration = 3.0
batch_size = 32

sequence = SoundSequence(path, sr=sr, duration=duration, batch_size=batch_size)

In [8]:
epochs = 100
autoencoder.fit(sequence, epochs=epochs)

Epoch 1/100


NameError: ignored